In [0]:
!pip install spacy

In [0]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [0]:
#Convert .tsv file to dataturks json format. 
import json
import logging
import sys
def tsv_to_json_format(input_path,output_path,unknown_label):
    try:
        f=open(input_path,'r') # input file
        fp=open(output_path, 'w') # output file
        data_dict={}
        annotations =[]
        label_dict={}
        s=''
        start=0
        for line in f:
            if line[0:len(line)-1]!='.\tO':
                word,entity=line.split('\t')
                s+=word+" "
                entity=entity[:len(entity)-1]
                if entity!=unknown_label:
                    if len(entity) != 1:
                        d={}
                        d['text']=word
                        d['start']=start
                        d['end']=start+len(word)-1  
                        try:
                            label_dict[entity].append(d)
                        except:
                            label_dict[entity]=[]
                            label_dict[entity].append(d) 
                start+=len(word)+1
            else:
                data_dict['content']=s
                s=''
                label_list=[]
                for ents in list(label_dict.keys()):
                    for i in range(len(label_dict[ents])):
                        if(label_dict[ents][i]['text']!=''):
                            l=[ents,label_dict[ents][i]]
                            for j in range(i+1,len(label_dict[ents])): 
                                if(label_dict[ents][i]['text']==label_dict[ents][j]['text']):  
                                    di={}
                                    di['start']=label_dict[ents][j]['start']
                                    di['end']=label_dict[ents][j]['end']
                                    di['text']=label_dict[ents][i]['text']
                                    l.append(di)
                                    label_dict[ents][j]['text']=''
                            label_list.append(l)                          
                            
                for entities in label_list:
                    label={}
                    label['label']=[entities[0]]
                    label['points']=entities[1:]
                    annotations.append(label)
                data_dict['annotation']=annotations
                annotations=[]
                json.dump(data_dict, fp)
                fp.write('\n')
                data_dict={}
                start=0
                label_dict={}
    except Exception as e:
        logging.exception("Unable to process file" + "\n" + "error = " + str(e))
        return None

tsv_to_json_format("/content/ner_dataset_260.tsv",'/content/ner_corpus_260.json','abc')

In [0]:
# Convert json file to spaCy format.
import plac
import logging
import argparse
import sys
import os
import json
import pickle

@plac.annotations(input_file=("ner_corpus_260.json", "option", "i", str), output_file=("spacy_corpus_260", "option", "o", str))

def main(input_file=None, output_file=None):
    try:
        training_data = []
        lines=[]
        with open(input_file, 'r') as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data['content']
            entities = []
            for annotation in data['annotation']:
                point = annotation['points'][0]
                labels = annotation['label']
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point['start'], point['end'] + 1 ,label))


            training_data.append((text, {"entities" : entities}))

        print(training_data)

        with open(output_file, 'wb') as fp:
            pickle.dump(training_data, fp)

    except Exception as e:
        logging.exception("Unable to process " + input_file + "\n" + "error = " + str(e))
        return None
if __name__ == '__main__':
    plac.call(main)

In [0]:
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("CVD is now the most common cause of death worldwide.")
for token in doc:
    print(token.text, '-->', token.dep_, '-->', token.head.text, '-->', token.head.pos_,
            '-->', [child for child in token.children])

CVD --> nsubj --> is --> AUX --> []
is --> ROOT --> is --> AUX --> [CVD, now, cause, .]
now --> advmod --> is --> AUX --> []
the --> det --> cause --> NOUN --> []
most --> advmod --> common --> ADJ --> []
common --> amod --> cause --> NOUN --> [most]
cause --> attr --> is --> AUX --> [the, common, of, worldwide]
of --> prep --> cause --> NOUN --> [death]
death --> pobj --> of --> ADP --> []
worldwide --> advmod --> cause --> NOUN --> []
. --> punct --> is --> AUX --> []


In [0]:
displacy.render(doc, style='dep',jupyter=True)

In [0]:
import spacy
from spacy.symbols import nsubj, VERB

nlp = spacy.load("en_core_web_sm")
doc = nlp("CVD is now the most common cause of death worldwide.")

# Finding a verb with a subject from below — good
verbs = set()
for possible_subject in doc:
    if possible_subject.dep == nsubj and (possible_subject.head.pos == VERB or possible_subject.head.pos == 'AUX'):
        verbs.add(possible_subject.head)
print(verbs)

set()


In [0]:
import spacy
from spacy.symbols import nsubj, VERB

nlp = spacy.load("en_core_web_sm")
doc = nlp("CVD is now the most common cause of death worldwide.")

def find_triples(root):
  if root.children == '':
    print(root.text,root.ent_type_)
  if root.children != '':
    for a in root.children:
      find_triples(a)
      print(a.text,a.ent_type_)
      if root.dep_ == 'ROOT':
        print()
# Finding a verb with a subject from below — good

for possible_subject in doc:
    if possible_subject.dep_ == 'ROOT':
      print(possible_subject.text,possible_subject.ent_type_)
      print()
      find_triples(possible_subject)


is 

CVD ORG

now 

the 
most 
common 
death 
of 
worldwide 
cause 

. 



In [0]:
import spacy
from spacy.symbols import nsubj, VERB

nlp = spacy.load("en_core_web_sm")

def find_triples(root):
  if root.children == '':
    print(root.text,root.ent_type_)
  if root.children != '':
    for a in root.children:
      find_triples(a)
      print(a.text,a.ent_type_)
      if root.dep_ == 'ROOT':
        print()

file = open("screened_sentences.txt","r")
lines = file.readlines()
for line in lines:
  print(line)
  doc = nlp(line)
# Finding a verb with a subject from below — good

  for possible_subject in doc:
    if possible_subject.dep_ == 'ROOT':
      print(possible_subject.text,possible_subject.ent_type_)
      print()
      find_triples(possible_subject)

Cardiovascular disease is now the most common cause of death worldwide. 

is 

Cardiovascular 
disease 

now 

the 
most 
common 
death 
of 
worldwide 
cause 


 
. 

Typical symptoms of an underlying cardiovascular disease includes pain or pressure in the chest, pain or discomfort in the arms and hortness of breath.

includes 

Typical 
an 
underlying 
cardiovascular 
disease 
of 
symptoms 

or 
pressure 
the 
, 
or 
discomfort 
pain 
chest 
in 
the 
and 
hortness 
breath 
of 
arms 
in 
pain 


 
. 

Cardiovascular disease is a class of diseases that involve the heart or blood vessels.

is 

Cardiovascular 
disease 

a 
that 
the 
or 
blood 
vessels 
heart 
involve 
diseases 
of 
class 


 
. 

Cardiovascular disease includes coronary artery diseases such as angina and myocardial infarction.

includes 

Cardiovascular 
disease 

coronary 
artery 
such 
and 
myocardial 
infarction 
angina 
as 
diseases 


 
. 

Other Cardiovascular diseases include stroke, heart failure, hypertensive h

'nlp = spacy.load("en_core_web_sm")\ndoc = nlp("Daily sugary drinks raise heart disease risk in males.")\n\nfor ent in doc.ents:\n    print(ent.text, ent.start_char, ent.end_char, ent.label_)'